# How to optimize target VM costs when moving them to Azure

> This thing works in  [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/eosfor/scripting-notes/HEAD)

## The scenario

Do you feel it is time to move your servers from on-prem DC to Azure cloud? One of the steps in this long journey is to map existing servers to their Azure counterparts. Along with that, we want to reduce the costs of the overall bundle of boxes. But at the same time, we need to get the highest performance possible. In theory, we can simply map the source VMs to the target VMs by the number of virtual CPUs and the amount of RAM. It is not the most difficult task in the world. However, in this case, we can't guarantee the highest possible performance for the lowest possible price. Suppose we always pick a VM with the lowest price, but in this case, we lose the performance. On the other hand, if we start picking VMs with the highest performance, we will also get the highest cost. Is there a middle ground? Is it even possible to get what we want? Let us see

But before we start, we need to invoke some .NET Interactive magic to let [mermaid](https://mermaid-js.github.io/mermaid/#/./flowchart?id=flowcharts-basic-syntax) and [Microsoft Dataframe](https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/) work. They will help us to represent data more conveniently

In [ ]:
#r "nuget:Microsoft.Data.Analysis,0.21.0-preview.23511.1"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab, *-*"

In [ ]:
Install-Module -Name Minizinc
Import-Module Minizinc -Force

## Minizinc

To do all that we need to start from minizinc. There is what they say about it on the [web site](https://www.minizinc.org/)

> MiniZinc is a free and open-source **constraint modeling language**.
>
> You can use MiniZinc to model constraint satisfaction and optimization problems in a **high-level**, **solver-independent** way, taking advantage of a large library of pre-defined constraints. Your model is then compiled into FlatZinc, a solver input language that is understood by a wide range of solvers.
>
> MiniZinc is developed at [Monash University](http://www.monash.edu/) in collaboration with [Data61 Decision Sciences](https://research.csiro.au/data61/tag/decision-sciences/) and the [University of Melbourne](http://unimelb.edu.au/).

In simple words, on one hand, it is a special **declarative** programming language, which is used to describe and solve a specific class of problems. On the other hand, it is an abstraction on top of a set of `solvers` which allows us to write a piece of code once and then try to solve a problem using different solvers.

In our case, we are solving the constraint satisfaction and optimization problem using minizinc.

## What we are going to do

In this section we perform a quick experiment, look at the result and explain it. The next section contains some high-level details on how all this works. Please note, for all this to work directly on the Windows system you may need to download and install [minizinc](https://www.minizinc.org/). Please also make sure that `minizinc.exe` is on the **PATH**.

> To hide some complexity we moved some additional logic [here](./vm-optimization-minizinc/helperFunctions.ps1). We will touch ubon it in the next section

Lets make it simple at first. Our solution contains three major parts:

- the [minizinc model](./vm-optimization-minizinc/vmCostsCalculation-integer.mzn), written in minizinc
- the [input data](./vm-optimization-minizinc/vmData.dzn) for the model
- [a few scripts](./vm-optimization-minizinc/helperFunctions.ps1), to simplify the look at feel of the tool

The input data contains he following:

- a piece of data about the source set of servers, their CPU, RAM and Disk
- a piece of data about Azure VMs in eastus2 region, exported from Azure
- a piece of data about Azure disks, also exported from Azure

Te model attempts to do all the heavy lifting **by itself**. It attempts to map each source server to its coud counterpart, but at the same time it follows the applied constraints, that we provide as part of the model. For example, our model demands that the amout of RAM of the cloud VM has to be not less than it is in the source box

In the best case scenario the solver guaratees that the solution is optimal.

To try all that, let us import all the necessary helpers - [dot sourcing](https://docs.microsoft.com/en-us/powershell/module/microsoft.powershell.core/about/about_scripts?view=powershell-7.2#script-scope-and-dot-sourcing)

In [ ]:
. .\vm-optimization-minizinc\helperFunctions.ps1

We are going to run three different tests:

- simple test, where we optimize for costs and performance separately, to have some data to compare to
- `cost -> performance` test, where we first optimize for costs, then fix it and optimize for performance. It means that we first find the minimal price and then try to find what we can to increase performance without breaking the cost limit
- `performance -> cost` test, where we first optimize for performance, then fix it and optimize for costs. It means that we first find the maximum performance and then see if we can adjust a set of VMs so that cost reduces but without affecting the performance.

### Simple test - optimizing separately

Minimize the cost

In [ ]:
Start-MinizincVMOptimizationModel -MinimizeCosts | Select-Object totalPrice, totalACU

Miximize the performance

In [ ]:
Start-MinizincVMOptimizationModel -MaximizeACU  | Select-Object totalPrice, totalACU

As we see from these results, the price in the first case is significantly less than in the second case. I also want to highlight that the numbers show that doubling the performance costs us over three times more. Now the question is, can we do better?

### Minimize the cost for the price of performance

We use what Microsoft calls [Azure compute unit (ACU)](https://docs.microsoft.com/en-us/azure/virtual-machines/acu) to estimate the performance. We pull this value from Azure APIs, and in the future, I'll show you how to do it. In our experiments we are trying to maximize it, keeping the cost as low as we can

In this experiment, we first minimize the cost and then try to maximize the performance of the resulting set of target VMs

In [ ]:
$ret = Start-MinizincVMOptimizationModel -MinimizeCosts | Start-MinizincVMOptimizationModel -FixCostsMaximizeACU
$ret | Select-Object totalPrice, totalACU

Here we see that, compared to the first test, the cost is still minimal. But we managed to increase the overall performance a bit.

Let us see which VM sizes the model has picked for us. For that, we try the [DataFrame](https://devblogs.microsoft.com/dotnet/an-introduction-to-dataframe/). The .NET Interactive can display it conveniently, so it is just, readable. For that, we have to convert the resulting set of VMs into the CSV string and then pass it to the extension method. Well, it does not look like an extension method, but it is. Unfortunately, this is how PowerShell does it.

In [ ]:
$csv = ($ret.vmRecords | ConvertTo-Csv -NoTypeInformation) -join "`n"
$df = [Microsoft.Data.Analysis.DataFrame]::LoadCsvFromString($csv)
[Microsoft.DotNet.Interactive.Kernel]::display($df)

The thing to pay attention to here is the `targetVMACU` column. It is always equal to 100. I did not check but assume that this is what helped to increase the performance a bit

Actually, it is not necessary to use DataFrames. We can use HTML and the extension method, but if the table is big it shows the long list of rows, which is a bit inconvenient

In [ ]:
$view = $ret.vmRecords | ConvertTo-Html -Fragment
[Microsoft.DotNet.Interactive.Kernel]::HTML($view) | Out-Display

### Minimize cost without affecting performance

Now we want to try a different approach. We first maximize performance, pin it and then try to minimize the total price, by asking the model to re-shuffle VMs to see if it is possible

In [ ]:
$ret2 = Start-MinizincVMOptimizationModel -MaximizeACU | Start-MinizincVMOptimizationModel -FixACUMinimizeCosts
$ret2 | Select-Object totalPrice, totalACU

And here we go! We get the total price which is just slightly higher than the minimal one, but with doubled performance! That is the win, I think!

In [ ]:
$csv = ($ret2.vmRecords | ConvertTo-Csv -NoTypeInformation) -join "`n"
$df2 = [Microsoft.Data.Analysis.DataFrame]::LoadCsvFromString($csv)
[Microsoft.DotNet.Interactive.Kernel]::display($df2)

The model chose completely different VM sizes, which helped to lower the cost but without affecting the performance

## How it really works

This section is for those who what to peek under the hood. But if you really want to dig deeper, it is better to watch a few courses below:


- [Basic Modeling for Discrete Optimization](https://www.coursera.org/learn/basic-modeling)
- [Advanced Modeling for Discrete Optimization](https://www.coursera.org/learn/advanced-modeling)


The overall solution diagram (using mermaid) is below

In [ ]:
#!mermaid
flowchart LR
    data[minizinc data] --> pwsh[PowerShell Crescendo Wrapper]
    model[minizinc model] --> pwsh[PowerShell Crescendo Wrapper]
    pwsh -.calls.-> mz[minizinc.exe]
    mz --uses--> solver[solver GECODE]
    mz -.returns results.-> pwsh
    pwsh --parses and returns-->r[results]

    source["source servers details (CPU, RAM, Disks)"] -.-> data
    azureVM["Azure VM pricing and ACU"]  -.-> data
    azureDisk["Azure Disk prices"]  -.-> data

The main piece of the solution is a [model](./vm-optimization-minizinc/minimize-totalPrice.mzn). As I mentioned before, the model uses a declarative approach to describe what we need. And that is the beauty of it. We don't need to tell it how to find a solution. We define a model in a form of variables we want to find, constraints, and objective functions. And the model finds a solution for us by itself. Let us see how it is done in our case:

The first constraint of the model requires that for all source servers we want to find such a target counterpart, that the amount  of RAM on it is more or equal to the source VM

```console
constraint forall(vm in existingVMs)(
    vmSizeRAM[selectedSize[vm]] >= vmRAM[vm]
);
```


The second constraint for the CPU is similar, but in this case, we allow it to be lower by 20%. It does not mean that all VMs are going to have fewer CPUs, it is just the lower boundary, to save costs a bit.

```console
constraint forall(vm in existingVMs)(
   vmSizeCPU[selectedSize[vm]] >=  vmCPU[vm] * 0.8
);
```

Besides that, we have two objective functions, the total price, and the total ACU. Each of them is just a sum of the corresponding values of all target VMs which our model is going to pick

```console
var int: totalPrice = sum(vm in existingVMs)(vmSizePrice[selectedSize[vm]]);
var int: totalACU = sum(vm in existingVMs)( vmSizeACU[selectedSize[vm]] );
```

And the last, but not least - `decision variables`, actually what we are looking for. Here we simply say that for each VM in the input array we want to find a corresponding VM size in the cloud, in a way that satisfies our constraints. Basically, we define an array of `decision variables` which our solver will try to calculate for us.

```console
array[existingVMs] of var vmSizes: selectedSize;
```

For all to work, the model lacks a statement that should tell which of the objective functions we want to optimize and how. For that, there is the `solve` keyword. There are a few options here:

- `solve satisfy` - seeks for any/all solutions which satisfy given constraints
- `solve minimize ...` - tries to minimize the objective function and satisfy constraints
- `solve maximize ...` - tries to maximize the objective function and satisfy constraints

On our implementation, the `Start-MinizincVMOptimizationModel` cmdlet plugs the correct `solve` operator into the model and runs it by calling minizinc

However, by default minizinc does not have a syntax to optimize for multiple parameters at a time. So we need to run the model twice. First, we run the model optimizing for one function, then get the result, supply it back to the model as a constraint and try optimizing for the second function. That is exactly what this pipeline does `Start-MinizincVMOptimizationModel -Costs | Start-MinizincVMOptimizationModel -Performance`


1. Plug `solve  minimize totalPrice;` to the model
2. The model executes, PowerShell takes the output from minizinc, parses it, and passes to the next command via PowerShell pipeline
3. This input gets transformed into a constraint + another `solve` statement - `constraint totalACU >= $($InputObject.totalACU); solve  minimize totalPrice;` or `constraint totalPrice <= $($InputObject.totalPrice * 10000); solve  maximize totalACU;` where `$($InputObject.xxx)` is theresult from the previous command
4. The model executes, PowerShell takes the output from minizinc, parses it, and passes to the next command via PowerShell pipeline

In the end, here is what is happening. The input to the model is the set of VMs with their sizes and all possible target VM sizes along with their attributes and ACU from Azure. For each VM the model tries to find the counterpart from the list of sizes such that it satisfies all constraints and at the same time tries to optimize the objective function.